## Import Dependency

In [2]:
#Gym dependency
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
#Helpe Stuff
import numpy as np
import random
import os
#Baseline stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

## Types of spaces

In [4]:
Discrete(3)     #will give 3 discrete values 0,1,2 at random

Discrete(3)

In [8]:
Discrete(3).sample()

2

In [10]:
Box(0,1,shape=(3,3))      #get a 3*3 array with high value 1 and low value 0

Box(0.0, 1.0, (3, 3), float32)

In [11]:
Box(0,1,shape=(3,3)).sample()

array([[0.22133914, 0.4925711 , 0.42720932],
       [0.06735376, 0.32599935, 0.643462  ],
       [0.12474088, 0.9579485 , 0.754942  ]], dtype=float32)

In [12]:
Tuple((Discrete(3),Box(0,1,shape=(3,)) ))    #can combine different spaces

Tuple(Discrete(3), Box(0.0, 1.0, (3,), float32))

In [13]:
Tuple((Discrete(3),Box(0,1,shape=(3,)) )).sample()

(0, array([0.5513666, 0.7873774, 0.7462026], dtype=float32))

In [14]:
Dict({'height':Discrete(3),'speed':Box(0,1,shape=(3,3)) })    #can combine different spaces

Dict('height': Discrete(3), 'speed': Box(0.0, 1.0, (3, 3), float32))

In [15]:
Dict({'height':Discrete(3),'speed':Box(0,1,shape=(3,3)) }).sample()

OrderedDict([('height', 1),
             ('speed',
              array([[0.11634909, 0.7965772 , 0.94389063],
                     [0.94095767, 0.3091148 , 0.38286665],
                     [0.5851955 , 0.57385063, 0.1289544 ]], dtype=float32))])

In [22]:
MultiBinary(3)          #will give 3 binnary outputs

MultiBinary(3)

In [19]:
MultiBinary(3).sample()

array([1, 0, 1], dtype=int8)

In [34]:
MultiDiscrete([5,2,2])           #first value vary between 0-4, second 0-1 etc

MultiDiscrete([5 2 2])

In [35]:
MultiDiscrete([5,2,2]).sample()

array([4, 0, 1], dtype=int64)

## Build an environment

In [38]:
#General idea of a Environment
class ShowerEnv(Env):
    def __inti__(self):
        pass
    def step(self,action):
        pass
    def render(self):
        pass
    def reset(self):
        pass

In [40]:
Box(low=0,high=100,shape=(1,)).sample()

array([35.837467], dtype=float32)

In [77]:
#General idea of a Environment
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [78]:
env=ShowerEnv()

In [79]:
env.observation_space

Box(0.0, 100.0, (1,), float32)

In [80]:
env.observation_space.sample()

array([39.349636], dtype=float32)

In [81]:
env.action_space

Discrete(3)

In [82]:
env.action_space.sample()

0

## Testing the environment

In [83]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:20
Episode:2 Score:-14
Episode:3 Score:-26
Episode:4 Score:-56
Episode:5 Score:0


In [84]:
log_path=os.path.join('Training','Logs')
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [87]:
model.learn(total_timesteps=40000)

Logging to Training\Logs\PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -32      |
| time/              |          |
|    fps             | 809      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -29.5       |
| time/                   |             |
|    fps                  | 534         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012764387 |
|    clip_fraction        | 0.0792      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 3.81e-05    |

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 60        |
|    ep_rew_mean          | -20.7     |
| time/                   |           |
|    fps                  | 432       |
|    iterations           | 11        |
|    time_elapsed         | 52        |
|    total_timesteps      | 22528     |
| train/                  |           |
|    approx_kl            | 0.0050299 |
|    clip_fraction        | 0.0187    |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.09     |
|    explained_variance   | 0.00048   |
|    learning_rate        | 0.0003    |
|    loss                 | 41.3      |
|    n_updates            | 140       |
|    policy_gradient_loss | -0.000733 |
|    value_loss           | 67        |
---------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -15.4 

In [88]:
shower_path=os.path.join('Training','Saved Models','Shower_Model_PPO')

In [89]:
model.save(shower_path)

In [90]:
del model

In [91]:
model=PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [95]:
evaluate_policy(model,env,n_eval_episodes=10,render=False)

(59.6, 0.7999999999999999)